In [1]:
import pickle
import numpy as np
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import glob
from esfMRI import stats_tests
from statsmodels.stats import nonparametric

In [2]:
with open("graph_theory/graphs.pkl", "rb") as f:
    graphs = pickle.load(f)

In [3]:
keys = ['k_core', 'clustering', 'modularity', 'degree_assortativity_coefficient', 'global_efficiency', 'local_efficiency', 'ratio']

In [22]:
# 对每个参数整体应用统计学分析
for key in keys:
    measures_preop = []
    measures_postop = []
    for sub in graphs:
        for run, graph in graphs[sub]["ses-preop"].items():
            if key == "clustering":
                measures_preop.append(sum(graph[key].values())/116)
            elif not np.isnan(graph[key]):
                measures_preop.append(graph[key])
        for run, graph in graphs[sub]["ses-postop"].items():
            if key == "clustering":
                measures_postop.append(sum(graph[key].values())/116)
            elif not np.isnan(graph[key]):
                measures_postop.append(graph[key])
    p_values = stats_tests(measures_preop, measures_postop, False)
    #     print(p_values)
    # else:
    print(key)
    if np.any(p_values[4:5] > 0.05):
        print("无明显变化")
    print(sum(measures_preop)/len(measures_preop), sum(measures_postop)/len(measures_postop))
    print(nonparametric.rank_compare_2indep(measures_preop, measures_postop).tuple)
    print()

k_core
18.271186440677965 10.78225806451613
(3.5777571793472034, 0.0006278687544275844)

clustering
0.5036242340701673 0.42831556601054516
(3.3542868301825366, 0.0012630603700364226)

modularity
0.3969259107538807 0.5395620434183688
(-4.357498039065151, 4.4915576487471325e-05)

degree_assortativity_coefficient
无明显变化
0.3610935037244088 0.37301312164720424
(-0.4412944397452481, 0.6596977536935209)

global_efficiency
0.35891723304123 0.23567027091724752
(5.03830539575699, 3.6644414622402904e-06)

local_efficiency
0.6089266852989381 0.5168289154232767
(3.8663531002476645, 0.00024188659625942513)

ratio
2.0602794109057267 2.84975460445315
(-6.270066178177548, 2.3912170573460824e-08)

